# E-MOSAIC

引用自[E. R. Q. Fernandes, A. C. P. L. F. de Carvalho and X. Yao, "Ensemble of Classifiers Based on Multiobjective Genetic Sampling for Imbalanced Data," in IEEE Transactions on Knowledge and Data Engineering, vol. 32, no. 6, pp. 1104-1115, 1 June 2020, doi: 10.1109/TKDE.2019.2898861.]


## 数据集的预处理 

In [1]:
from utils.dataset_utils import get_distribution
from instance_selection.parameter.parameter import *  # 导入参数的设定
from instance_selection.operator.init_toolbox import init_toolbox_emosaic
from instance_selection.operator.metrics import calculate_gmean_mauc, calculate_average_gmean_mauc
from instance_selection.operator.genetic_operator import selTournamentNDCD
from instance_selection.operator.ensemble import vote_result_ensembles, ensemble_individuals
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import scipy.io as sio  # 从.mat文件中读取数据集
import random
from deap import tools
import warnings

warnings.filterwarnings("ignore")  # 忽略警告

DATASET = Contraceptive  # 数据集名称（包含对应参数的字典形式）
datasetname = DATASET['DATASETNAME'].split('.')[0]

# 加载、划分数据集
mat_data = sio.loadmat('../../data/dataset/' + DATASET['DATASETNAME'])
x = mat_data['X']
y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=RANDOM_SEED)  # 划分数据集
scaler = StandardScaler()  # 数据的标准化
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

unique_elements_all, classes_all, counts_all = get_distribution(y)  # 获取原始数据集分布
unique_elements_train, classes_train, counts_train = get_distribution(y_train)  # 获取训练集分布
unique_elements_test, classes_test, counts_test = get_distribution(y_test)  # 获取测试集分布
print(datasetname + f' distribution: {counts_all}')
print(f'trainset distribution: {counts_train}')
print(f'testset distribution: {counts_test}')
model = MLPClassifier(hidden_layer_sizes=(DATASET['HIDDEN_SIZE'],), max_iter=DATASET['MAX_ITER'],
                      random_state=RANDOM_SEED, learning_rate_init=DATASET['LEARNING_RATE'])

Contraceptive distribution: [629 333 511]
trainset distribution: [499 262 417]
testset distribution: [130  71  94]


## E-MOSAIC

In [2]:
toolbox = init_toolbox_emosaic(model, x_train, y_train, n_splits=N_SPLITS, random_seed=RANDOM_SEED)  # 初始化toolbox


def main(x_train, y_train, model, balanced_method='balanced'):
    not_replaced = 0
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    logbook = tools.Logbook()
    logbook.header = "gen", "fronts", "ensembles_size", "avg_gmean", "avg_mauc"
    ####################################种群的初始化###########################
    pop = toolbox.population(n=POPSIZE)  # 个体编码默认全为0
    pop = toolbox.init_population(pop, balanced_method=balanced_method)  # 初始化种群中的个体
    toolbox.evaluate(pop) # 计算个体的适应度
    save_ensembles = pop  # 保存ensembles为初始种群
    save_gmean, save_mauc = calculate_average_gmean_mauc(pop)
    ####################################种群的迭代#################################################
    for gen in range(1, NGEN + 1):
        offspring = selTournamentNDCD(pop, POPSIZE, tournsize=3)  # 锦标赛选择（1、先根据非支配排序的等级2、再根据拥挤距离）
        offspring = [toolbox.clone(ind) for ind in offspring]
        for i in range(0, len(offspring) - 1, 2):
            if random.random() <= CXPB:
                offspring[i], offspring[i + 1] = toolbox.mate(offspring[i], offspring[i + 1])  # 单点交叉
            offspring[i] = toolbox.mutate(offspring[i], MR)[0]  # 二进制反转突变
            offspring[i + 1] = toolbox.mutate(offspring[i + 1], MR)[0]  # 二进制反转突变
            del offspring[i].fitness.values, offspring[i + 1].fitness.values
        #############################################################合并、去重#####################################################
        pop = pop + offspring  # 种群的合并
        pop, _ = toolbox.remove_duplicates(pop)  # 去重
        while len(pop) < POPSIZE:  # 保证种群大小为POPSIZE
            add_individual = []
            num_add = POPSIZE - len(pop)
            for i in range(0, num_add):
                index = random.randint(0, len(offspring) - 1)  # 在0-len(offspring)范围内随机产生一个索引
                offspring[index] = toolbox.mutate(offspring[index], MR)[0]  # 选择index对应的个体进行突变
                del offspring[index].fitness.values
                add_individual.append(offspring[index])
            pop = pop + add_individual  # 种群的合并
            pop, _ = toolbox.remove_duplicates(pop)  # 去重
        toolbox.evaluate(pop) # 计算新种群适应度
        ###############################################得到pareto_fronts############################################
        pop, pareto_fronts = toolbox.select(pop, POPSIZE)
        ensembles = pop  # 集成的个体
        curr_gmean, curr_mauc = calculate_average_gmean_mauc(ensembles)
        if curr_gmean >= save_gmean and curr_mauc >= save_mauc:
            save_ensembles = ensembles
            not_replaced = 0
        else:
            not_replaced += 1
        if not_replaced >= STOP_SIGN:
            break  # 迭代结束
        record = stats.compile(pop)
        logbook.record(gen=gen, fronts=len(pareto_fronts), ensembles_size=len(ensembles),
                       avg_gmean=curr_gmean, avg_mauc=curr_mauc, **record)
        print(logbook.stream)
    ensemble_classifiers = ensemble_individuals(save_ensembles, model, x_train, y_train)
    return ensemble_classifiers


if __name__ == "__main__":
    print("*****************算法开始执行：******************")
    ensemble_classifiers = main(x_train, y_train, model=model)
    vote_pred_prob = vote_result_ensembles(ensemble_classifiers, x_test)  # 默认预测结果是软标签
    gmean, mauc, recall_per_class = calculate_gmean_mauc(vote_pred_prob, y_test)
    print(f"Reacll:{recall_per_class}，Gmean：{gmean}，mAUC：{mauc}")
    print("*****************算法执行结束！******************")

*****************算法开始执行：******************
gen	fronts	ensembles_size	avg_gmean	avg_mauc
1  	10    	40            	0.515382 	0.689041
2  	10    	40            	0.519915 	0.693497
3  	8     	40            	0.524123 	0.697774
4  	9     	40            	0.529005 	0.701436
5  	11    	40            	0.53255  	0.70556 
6  	11    	40            	0.535159 	0.708665
7  	11    	40            	0.535559 	0.708966
8  	10    	40            	0.539175 	0.709955
9  	10    	40            	0.540403 	0.711573
10 	9     	40            	0.54195  	0.712257
11 	10    	40            	0.543487 	0.714058
12 	9     	40            	0.54473  	0.716549
13 	9     	40            	0.546199 	0.717297
14 	9     	40            	0.546137 	0.719787
15 	9     	40            	0.54658  	0.719969
16 	9     	40            	0.546611 	0.721153
17 	8     	40            	0.547878 	0.721578
18 	8     	40            	0.548093 	0.723234
19 	8     	40            	0.548118 	0.724106
20 	7     	40            	0.547749 	0.725012
21 	7     	4